#### Day 4 
_Anirudh Sathish CS20B1125_
###### Perform LED digit classification using Naive bayes classifier  in python.

In [477]:
#  Importing neccesary libraries 
import pandas as pd
import numpy as np
import random 
import copy 
from sklearn.utils import shuffle

In [478]:
# function to give accuracyScore of prediction 
def accuracyScore(Y_predicted,Y_true):
    positive = 0
    total = 0 
    for i in range(len(Y_predicted)):  
        if(Y_predicted.iloc[i][0] == Y_true.iloc[i][0]):
            positive+=1
        total+=1
    pred = positive/total
    return pred 

In [479]:
led = pd.read_csv("led.csv")

In [480]:
print(led)

   s1  s2  s3  s4  s5  s6  s7  Digit-ID
0   0   1   1   0   0   0   0         1
1   1   1   0   1   1   0   1         2
2   1   1   1   1   0   0   1         3
3   0   1   1   0   0   1   1         4
4   1   0   1   1   0   1   1         5
5   0   0   1   1   1   1   1         6
6   1   1   1   0   0   0   0         7
7   1   1   1   1   1   1   1         8
8   1   1   1   0   0   1   1         9
9   1   1   1   1   1   1   0         0


In [481]:
# Now Generate training data 
# Let us set a certain noise level 
noise_level = 0.1
# This means that with a probability 0.1 the digit might flip 

# Let us augument 20 values for each digit 
# This is including its original value  

#Sepearate out the input and the ouput parts of the led
led_output = led.iloc[:,-1]
input_bool = [True for i in range(7)]
input_bool.append(False)
led_input = led.iloc[:,input_bool]


In [482]:
led_save = copy.deepcopy(led)

In [483]:
def augument(number,led,led_output):
    dataAugumentationValue = number
    for t in range(len(led_output)):
        resu = []
        dig1 = led_input.iloc[t]
        for digit in dig1:
            myValue = str(digit)
            notMyValue = str(int(not(digit)))
            combination = notMyValue+myValue
            resultant = random.choices(combination, cum_weights=(noise_level, 1.00), k = dataAugumentationValue)
            resultant_int = [int(d) for d in resultant]
            resu.append(resultant_int)
        output = [led_output.iloc[t] for i in range(dataAugumentationValue)]
        resu.append(output)
        res = np.array(resu).transpose()
        augumented_dig = pd.DataFrame(res)
        augumented_dig.columns = led.columns
        led = pd.concat([led,augumented_dig])
    return led

In [484]:
led = augument(19,led,led_output)
print(led.shape)

(200, 8)


In [485]:
led_test = shuffle(augument(5,led_save,led_output),random_state=20)
led_test_inp = led_test.iloc[:,input_bool]
led_test_op = led_test.iloc[:,-1]

In [486]:
# Now led contains our training set values
# Let us train our dataset on naive Baiyes classifier 
print(led.shape)

(200, 8)


In [487]:
# Now let us shuffle our dataset 
shuffled_train_led = shuffle(led,random_state=20)

In [488]:
# Now let us perform naive bayes classification training on this 
print(shuffled_train_led.shape)

(200, 8)


In [489]:
# Create a classifier class 
class NaiveBayesClassifier():
    def __init__(self):
        self.OutputLabelsHash = {}
        self.attributesHash = {}
    def convertFeatureSet(self,data):
        labels = set(data)
        values = []
        for i in range(len(labels)):
            values.append(0)
        resultant = {}
        for label in labels:
            for value in values:
                resultant[label] = value
                values.remove(value)
                break
        return resultant
    def getProb(self):
        pOfDigits = self.convertFeatureSet(self.OutputLabels)
        for i  in self.OutputLabels:
            pOfDigits[i] +=1
        self.digitCount = copy.deepcopy(pOfDigits)
        for i in range(len(pOfDigits)):
            pOfDigits[i] = pOfDigits[i]/len(self.OutputLabels)
        return  pOfDigits
    def giveTableAttributeKnowledge(self,digit):
        # Now work with the input features

    ## 10 -> The digit is obtained and the value was 0 
        possible_values = ["10","11"]
        possiblity_Set = self.convertFeatureSet(possible_values)
        #print(possiblity_Set)  
        for i in range(self.inputFeatures.shape[1]):
            # For s1,s2 and so on looping
            count = 0
            count_10 = 0 
            count_11 = 0 
            for j in self.inputFeatures.iloc[:,i]:
                # For possiblity 10 i.e digit obtained and value was 0
                # if obtained == 0 and output = digit
                if(j == 0 and self.OutputLabels.iloc[count] == digit):
                    count_10+=1
                # For possiblity 11 i.e digit obtained and value was 0
                # if obtained == 1 and output = digit
                if(j == 1 and self.OutputLabels.iloc[count] == digit):
                    count_11+=1
                
                count+=1
            possiblity_Set["10"] = round(((count_10+1)/(2+self.digitCount[digit])),2) 
            possiblity_Set["11"] = round(((count_11+1)/(2+self.digitCount[digit])),2)  
            string = "s"+str(i+1)
            self.AttributesUnique[string] = copy.deepcopy(possiblity_Set)
        self.Table[digit] = copy.deepcopy(self.AttributesUnique)
    def fitData(self,features,Outputlabel):
        # Make tables using these
        self.inputFeatures = features
        self.OutputLabels = Outputlabel
        self.Table = self.convertFeatureSet(self.OutputLabels)
        # Create the tables using dict
        self.AttributesUnique = self.convertFeatureSet(self.inputFeatures)
        # Later add for all digits
        self.digitProbability = self.getProb()

        #for all the digits find the table 
        for i in range(len(self.digitCount)): 
            self.giveTableAttributeKnowledge(i)
        #The data is trained 
    def predictDigit(self,inputTuple):
        # Given a pandas single tuple df , predict the value
        numberWiseProb = self.convertFeatureSet(self.OutputLabels)
        # Now find numberWise Probability
        # for that iterate over the numbers 
        numbers = set(self.OutputLabels)
        for number in numbers:
            # find the probability for the tuple
            #Iterate over each in tuple 
            prob_number = 1
            for i in range(len(inputTuple)):
                string = "s"+str(i+1)
                numberString = "1"+str(inputTuple[i])
                value = self.Table[number][string][numberString]
                prob_element = value
                prob_number*=prob_element
            numberWiseProb[number] = prob_number
        labelFinal = max(numberWiseProb, key=numberWiseProb.get)
        return labelFinal
    def predictData(self,input):
        labels = []
        input_tuple_header = input.columns
        for i in range(input.shape[0]):
            label = self.predictDigit(input.iloc[i])
            labels.append(label)
        labels = pd.DataFrame(labels)
        return labels
        
        


In [490]:
nB = NaiveBayesClassifier()
nB.fitData(shuffled_train_led.iloc[:,input_bool],shuffled_train_led.iloc[:,-1])

In [491]:
prediction = nB.predictData(led_test_inp)
print(prediction)

    0
0   5
1   3
2   2
3   0
4   0
5   1
6   0
7   6
8   5
9   5
10  8
11  3
12  4
13  2
14  9
15  2
16  4
17  0
18  4
19  0
20  5
21  4
22  1
23  1
24  3
25  0
26  0
27  1
28  4
29  9
30  0
31  0
32  1
33  4
34  7
35  8
36  8
37  6
38  7
39  8
40  8
41  2
42  2
43  7
44  5
45  3
46  7
47  5
48  5
49  8
50  7
51  1
52  3
53  0
54  0
55  4
56  5
57  2
58  1
59  6


In [492]:
print(accuracyScore(prediction,led_test_op.to_frame()))

0.7333333333333333


In [493]:
def segmentDisplay(SegmentNo):
    if SegmentNo[0] == 1 :
        print(" __ ")
    else :
        print("    ")

    if SegmentNo[1] ==1 and SegmentNo[5] == 1 :
        print("|  |")
    elif SegmentNo[1] == 0 and SegmentNo[5] == 1:
        print("|   ")
    elif SegmentNo[1] == 1 and SegmentNo[5] == 0:
        print("   |")
    elif SegmentNo[1] == 0 and SegmentNo[5] == 0:
        print("    ")
    if SegmentNo[6] == 1 :
        print(" -- ")
    else :
        print("    ")

    if SegmentNo[2] ==1 and SegmentNo[4] == 1 :
        print("|  |")
    elif SegmentNo[2] == 0 and SegmentNo[4] == 1:
        print("|   ")
    elif SegmentNo[2] == 1 and SegmentNo[4] == 0:
        print("   |")
    elif SegmentNo[2] == 0 and SegmentNo[4] == 0:
        print("    ")

    if SegmentNo[3] == 1 :
        print(" -- ")
    else :
        print("    ")
def numberDisplay(number):
    #Get tuple for the number from the correct result
    for i in range(led_save.shape[0]):
        if(led_save.iloc[:,-1][i] == number):
            #fetch the tuple
            tuple_val = led_save.iloc[:,input_bool].iloc[i]
            segmentDisplay(tuple_val)
            break


In [494]:
# Testing out what might be a faulty case
# picking a test input

#Before correction
print("Faulty")
lis = [1,0,0,1,0,1,1]
segmentDisplay(lis)

print("Correction")
liz = pd.DataFrame(lis)
predictedNumber = nB.predictData(liz.transpose())
numberDisplay(predictedNumber.iloc[0][0])

Faulty
 __ 
|   
 -- 
    
 -- 
Correction
 __ 
|   
 -- 
   |
 -- 
